In [2]:
import yfinance as yf
import pandas as pd 
import numpy as np 
from pathlib import Path


In [3]:
sp500 = yf.Ticker('^GSPC')

In [4]:
#get stock info
sp500.info

{'exchange': 'SNP',
 'shortName': 'S&P 500',
 'exchangeTimezoneName': 'America/New_York',
 'exchangeTimezoneShortName': 'EDT',
 'isEsgPopulated': False,
 'gmtOffSetMilliseconds': '-14400000',
 'quoteType': 'INDEX',
 'symbol': '^GSPC',
 'messageBoardId': 'finmb_INDEXGSPC',
 'market': 'us_market',
 'previousClose': 3900.11,
 'regularMarketOpen': 3913,
 'twoHundredDayAverage': 4403.2607,
 'trailingAnnualDividendYield': None,
 'payoutRatio': None,
 'volume24Hr': None,
 'regularMarketDayHigh': 3945.86,
 'navPrice': None,
 'averageDailyVolume10Day': 4731210000,
 'totalAssets': None,
 'regularMarketPreviousClose': 3900.11,
 'fiftyDayAverage': 4054.9153,
 'trailingAnnualDividendRate': None,
 'open': 3913,
 'toCurrency': None,
 'averageVolume10days': 4731210000,
 'expireDate': None,
 'yield': None,
 'algorithm': None,
 'dividendRate': None,
 'exDividendDate': None,
 'beta': None,
 'circulatingSupply': None,
 'startDate': None,
 'regularMarketDayLow': 3820.14,
 'priceHint': 2,
 'currency': 'USD'

In [5]:
# get historical market data
sp500_df = sp500.history(period='max')
sp500_df.head(10)


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000,0,0
1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000,0,0
1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000,0,0
1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000,0,0
1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000,0,0
1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000,0,0
1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000,0,0
1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000,0,0
1950-01-13,16.670000,16.670000,16.670000,16.670000,3330000,0,0


In [6]:
#seperating close and volume columns to their own dataframe 
sp500_close = sp500_df.drop(columns=['Dividends', 'Stock Splits', 'Low', 'High', 'Open', 'Volume'], axis=1)

sp500_volume = sp500_df.drop(columns=['Dividends', 'Stock Splits', 'Low', 'High', 'Open', 'Close'], axis=1)

display(sp500_close)
display(sp500_volume)


,Close
Date,
1950-01-03,16.660000
1950-01-04,16.850000
1950-01-05,16.930000
1950-01-06,16.980000
1950-01-09,17.080000
...,...
2022-06-22,3759.889893
2022-06-23,3795.729980
2022-06-24,3911.739990


,Volume
Date,
1950-01-03,1260000
1950-01-04,1890000
1950-01-05,2550000
1950-01-06,2010000
1950-01-09,2520000
...,...
2022-06-22,4342270000
2022-06-23,4078100000
2022-06-24,6742870000


In [7]:
#is this needed? 
#sp500_df.to_csv('S&P_500.csv',float_format='%.2f')

In [10]:
#Calculating the percent change of the 'Close' column to get the return as a %
sp500_close['pct_change'] = sp500_close['Close'].pct_change()
#dropping NaN values
sp500_df.dropna()
display(sp500_close)

,Close,pct_change
Date,,
1950-01-03,16.660000,NaN
1950-01-04,16.850000,0.011405
1950-01-05,16.930000,0.004748
1950-01-06,16.980000,0.002953
1950-01-09,17.080000,0.005889
...,...,...
2022-06-22,3759.889893,-0.001302
2022-06-23,3795.729980,0.009532
2022-06-24,3911.739990,0.030563


In [12]:
#Calculating percent change of the volume to easier see change while studying correlation between volume and price
sp500_volume['pct_change'] = sp500_volume['Volume'].pct_change()

display(sp500_volume)

,Volume,pct_change
Date,,
1950-01-03,1260000,NaN
1950-01-04,1890000,0.500000
1950-01-05,2550000,0.349206
1950-01-06,2010000,-0.211765
1950-01-09,2520000,0.253731
...,...,...
2022-06-22,4342270000,0.052774
2022-06-23,4078100000,-0.060837
2022-06-24,6742870000,0.653434


In [23]:
#setting variables for long and short MA 
short_ma = 5
long_ma = 100


#create columns for short_ma and long_ma
sp500_close['SMA_long'] = sp500_close['Close'].rolling(window = long_ma).mean()

sp500_close['SMA_short'] = sp500_close['Close'].rolling(window = short_ma).mean()



display(sp500_close.dropna())


,Close,pct_change,SMA_long,SMA_short
Date,,,,
1950-05-25,18.690001,0.000000,17.512800,18.674000
1950-05-26,18.670000,-0.001070,17.532900,18.672000
1950-05-29,18.719999,0.002678,17.551600,18.696000
1950-05-31,18.780001,0.003205,17.570100,18.710000
1950-06-01,18.770000,-0.000532,17.588000,18.726000
...,...,...,...,...
2022-06-22,3759.889893,-0.001302,4257.481506,3731.256006
2022-06-23,3795.729980,0.009532,4251.120305,3732.404004
2022-06-24,3911.739990,0.030563,4245.082207,3781.397998


In [ ]:
sp500_short = sp500_close.drop(columns=['SMA_long', 'pct_change', 'Close','Date'], axis=1)

In [ ]:
sp500_short=sp500_short.dropna()
sp500_short

In [ ]:
# Using the df_mercado_trends DataFrame, reset the index so the date information is no longer the index
sp500_short = sp500_short.reset_index()

# Label the columns ds and y so that the syntax is recognized by Prophet
sp500_short.columns = ["ds", "y"]

# Drop an NaN values from the prophet_df DataFrame
sp500_short = sp500_short.dropna()

# View the first and last five rows of the mercado_prophet_df DataFrame
display(sp500_short.head())
display(sp500_short.tail())